Ex. 5.8 - MLE and model selection for a 2d discrete distribution
------------------------------------------------------

In [2]:
import numpy as np
import matplotlib as plt
import scipy.stats as sps
import scipy.misc as spm
import scipy.optimize as spo
%matplotlib inline

x = np.array([1,1,0,1,1,0,0])
y = np.array([1,0,0,0,1,0,1])

In [11]:
def ll(th, x,y, npar_2=1):
    
    assert(len(x)==len(y))
    n = len(x)
    n_00 = sum((x==0)& (y==0))
    n_01 = sum((x==0)& (y==1))
    n_10 = sum((x==1)& (y==0))
    n_11 = sum((x==1)& (y==1))
    assert(n_00 + n_01 + n_10 + n_11 == n)
    comb = spm.factorial(n)/(spm.factorial(n_00)*spm.factorial(n_10)*spm.factorial(n_01)*spm.factorial(n_11))
    if npar_2:
        th1, th2 = th        
        res = comb * (th1 * th2)**n_11 * (th1 * (1-th2))**n_10 * ((1-th1) * (1-th2))**n_01 * ((1-th1)*th2)**n_00    
    else:
        th1, th2, th3 = th #th_11, th_10, th_01
        res = comb * th1**n_11 * th2**n_10 * th3**n_01 * (1-th1-th2-th3)**n_00 
    return res

def m_ll(th,x,y,npar_2 = 1):
    return -ll(th,x,y,npar_2)

In [29]:
def mle_2par(x,y):
    nx_1 = sum(x==1)
    ny_1 = sum(y==1)
    n = len(x)
    th_1_hat = nx_1 / float(n)
    th_2_hat = sum(x==y) / float(n)

    return (th_1_hat, th_2_hat)
print ll(mle_2par(x,y), x,y)
spo.minimize(m_ll, x0 = np.array([0.5,0.5]), args = (x,y,1,), bounds = ((0.,1.),(0.,1.)))['x']

0.0443787891107


array([ 0.57142622,  0.57142622])

In [28]:
def mle_4par(x,y):
    th_01 = sum((x==0)& (y==1)) / float(n)
    th_10 = sum((x==1)& (y==0)) / float(n)
    th_11 = sum((x==1)& (y==1)) / float(n)
    th_00 = 1 - th_01 - th_10 - th_11

    return (th_11, th_10, th_01)

print ll(mle_4par(x,y),x,y,0)
spo.minimize(m_ll, x0 = np.array([0.25,0.25,0.25]), args = (x,y,0,), bounds = ((0.,1.),(0.,1.), (0.,1.)))['x']

0.0489591921733


array([ 0.28571313,  0.28571313,  0.1428568 ])

In [33]:
L_4 = 0
L_2 = 0
for i in range(len(x)):
    x_t = np.delete(x, i)
    y_t = np.delete(y, i)
    L_2 += ll(mle_2par(x_t,y_t),x,y)
    L_4 += ll(mle_4par(x_t,y_t),x,y,0)
    
print 'L_4 = ', L_4
print 'L_2 = ', L_2

L4 =  0.208076566737
L2 =  0.254656860189


In [38]:
BIC_4 = ll(mle_4par(x,y),x,y,0) - 1.5 * np.log(len(x))
BIC_2 = ll(mle_2par(x,y),x,y) - 1. * np.log(len(x))

print 'BIC 2 par = ', BIC_2
print 'BIC 4 par = ', BIC_4

BIC 2 par =  -1.90153135994
BIC 4 par =  -2.86990603141
